In [4]:
import os
import json
import pandas as pd
import numpy as np
import jinja2
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")
pd.set_option('display.max_colwidth', 100)  # default is 50

In [ ]:
# List of root directories to process
root_dirs = [
    "experiments/1_cost_assessment_per_model_no_defs/results/2025-06-08_20-28-18",
    "experiments/1_cost_assessment_per_model_no_defs/results/2025-06-08_21-04-01",
    "experiments/1_cost_assessment_per_model/results/2025-06-08_20-13-16",
    "experiments/1_cost_assessment_per_model/results/2025-06-08_20-58-45"
]

# Prepare a list to collect all the JSON data
data_list = []

# Walk through each directory
for root_dir in root_dirs:
    for subdir, _, files in os.walk(root_dir):
        for file in files:
            if file.endswith('.json'):
                json_path = os.path.join(subdir, file)
                with open(json_path, 'r') as f:
                    try:
                        data = json.load(f)
                        data_list.append(data)
                    except json.JSONDecodeError:
                        print(f"Failed to decode JSON: {json_path}")

# Convert to a DataFrame
df = pd.DataFrame(data_list)

# Define models to exclude
exclude_models = ['openai/o4-mini-high', 'openai/gpt-4o-mini']

# Drop rows where model is in exclude_models
df = df[~df['model'].isin(exclude_models)]

# Example: check number of rows
len(df)


80

In [23]:
df.head(6).sort_values(by=["run_id", "attempt"])

,model,exercise,prompt,tutorial,full_input,exercise_content,output,token_count,input_tokens,thinking_tokens,output_tokens,cost,thinking_mode,errors,line_with_error,success,run_id,attempt,max_attempts
2,anthropic/claude-3.7-sonnet:thinking,3_11_4,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...,"Your proof seems to have some mistakes in it, as it does not compile correctly.\nHere is the err...",Section Exercise_3_11_4.\n\nVariable f : ℝ → ℝ.\nVariable g : ℝ → ℝ.\n\nLemma exercise_3_11_4 :\...,"Proof.\nAssume that (∀ M1 ∈ ℝ, ∃ y > 0, ∀ x > y, f(x) > M1) (H1).\nAssume that (∀ M2 < 4, ∃ z > ...",12279,6288,5477,514,0.108729,True,"File ""/tmp/E3_11_4_hdj_wc2l.v"", line 59, characters 0-1:\nError: [Focus] Wrong bullet *: Current...","* Indeed, (c ∈ ℝ).",False,anthropic/claude-3.7-sonnet:thinking::3_11_4::1,2,2
4,anthropic/claude-3.7-sonnet:thinking,4_9_3,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...,"Your proof seems to have some mistakes in it, as it does not compile correctly.\nHere is the err...","Lemma ex_4_9_3 : 4 is the supremum of [0,4).\nProof.","Proof.\nWe need to show that (4 is the supremum of [0,4)).\n\nFirst, we show that 4 is an upper ...",9024,5894,2915,215,0.064632,True,"File ""/tmp/E4_9_3_il3gjirm.v"", line 37, characters 22-24:\nError:\nSyntax error: [ltac_use_defau...","First, we show that 4 is an upper bound for [0,4).",False,anthropic/claude-3.7-sonnet:thinking::4_9_3::1,2,2
0,deepseek/deepseek-chat-v3-0324,3_11_4,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...,"Your proof seems to have some mistakes in it, as it does not compile correctly.\nHere is the err...",Section Exercise_3_11_4.\n\nVariable f : ℝ → ℝ.\nVariable g : ℝ → ℝ.\n\nLemma exercise_3_11_4 :\...,Here's the corrected proof that properly follows Waterproof syntax:\n\nProof.\nAssume that (∀ M1...,4740,4402,0,338,0.001618,False,"File ""/tmp/E3_11_4___hb5mnp.v"", line 48, characters 11-15:\nError: Syntax error: ',' or 'accordi...","* Obtain y such that (y > 0) and (∀ x > y, f(x) > 7) (iii).",False,deepseek/deepseek-chat-v3-0324::3_11_4::1,2,2
1,deepseek/deepseek-chat-v3-0324,4_9_3,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...","Lemma ex_4_9_3 : 4 is the supremum of [0,4).\nProof.","Proof. \nWe need to show that (4 is the supremum of [0,4)). \nWe show both statements. \n* We...",3815,3630,0,185,0.001252,False,"File ""/tmp/E4_9_3_p_kt1ckz.v"", line 37, characters 0-24:\nError: This is not an 'and' statement,...",We show both statements.,False,deepseek/deepseek-chat-v3-0324::4_9_3::1,1,2
3,deepseek/deepseek-r1-0528,3_11_4,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",# Waterproof Tutorial\n\n## 1. We conclude that\n\n### Example:\n\nLemma example_reflexivity :\n...,"Your proof seems to have some mistakes in it, as it does not compile correctly.\nHere is the err...",Section Exercise_3_11_4.\n\nVariable f : ℝ → ℝ.\nVariable g : ℝ → ℝ.\n\nLemma exercise_3_11_4 :\...,The error occurred because the syntax for using a universal quantifier was incorrect. The correc...,10599,4414,5803,382,0.015505,True,"File ""/tmp/E3_11_4_6_kmowug.v"", line 47, characters 10-14:\nError: Syntax error: ',' or 'accordi...","Obtain z0 such that (z0 > 0) and (∀ x > z0, g(x) > 3) (H3).",False,deepseek/deepseek-r1-0528::3_11_

In [56]:
total_cost = df['cost'].sum()
total_cost

np.float64(2.28895363)

In [5]:
# Total success rate per model
summary = df.groupby('model')['success'].mean().reset_index()
summary['success'] = (summary['success'] * 100).round(2)  # Convert to percentage
summary.rename(columns={'success': 'Success Rate (%)'}, inplace=True)

# Export to LaTeX
latex_table = summary.to_latex(index=False, column_format="ll", caption="Success rate per model", label="tab:success_rate")
print(latex_table)

\begin{table}
\caption{Success rate per model}
\label{tab:success_rate}
\begin{tabular}{ll}
\toprule
model & Success Rate (%) \\
\midrule
openai/o4-mini & 18.180000 \\
x-ai/grok-3-mini-beta & 16.670000 \\
\bottomrule
\end{tabular}
\end{table}



In [6]:
# Total success rate per exercise (check if this code is correct)
summary = df.groupby('exercise')['success'].mean().reset_index()
summary['success'] = (summary['success'] * 100).round(2)  # Convert to percentage
summary.rename(columns={'success': 'Success Rate (%)'}, inplace=True)

# Export to LaTeX
latex_table = summary.to_latex(index=False, column_format="ll", caption="Success rate per exercise", label="tab:success_rate")
print(latex_table)

\begin{table}
\caption{Success rate per exercise}
\label{tab:success_rate}
\begin{tabular}{ll}
\toprule
exercise & Success Rate (%) \\
\midrule
11_not_closed & 0.000000 \\
3_11_2 & 36.360000 \\
\bottomrule
\end{tabular}
\end{table}



In [57]:
# Grid of success rates per model and per exercise (requires multiple runs of all models)
grid = df.pivot_table(index='model', columns='exercise', values='success', aggfunc='mean') * 100
grid = grid.round(2)
grid

exercise,3_11_4,4_9_3
model,,
anthropic/claude-3.7-sonnet:thinking,0.0,0.0
anthropic/claude-sonnet-4,0.0,0.0
deepseek/deepseek-chat-v3-0324,0.0,0.0
deepseek/deepseek-r1-0528,0.0,0.0
google/gemini-2.5-flash-preview-05-20,0.0,0.0
google/gemini-2.5-flash-preview-05-20:thinking,0.0,0.0
openai/gpt-4.1,0.0,0.0
openai/gpt-4o-mini,0.0,0.0
openai/o4-mini,0.0,0.0


In [58]:
# Average output token counts per model
output_tokens_avg = df.groupby('model')['output_tokens'].mean().reset_index()
output_tokens_avg.rename(columns={'output_tokens': 'Avg Output Tokens'}, inplace=True)

# Average thinking token counts per model
thinking_tokens_avg = df.groupby('model')['thinking_tokens'].mean().reset_index()
thinking_tokens_avg.rename(columns={'thinking_tokens': 'Avg Thinking Tokens'}, inplace=True)

# Get the thinking_mode per model
thinking_mode_per_model = df[['model', 'thinking_mode']].drop_duplicates()

# Merge into the result
output_tokens_avg = output_tokens_avg.merge(thinking_mode_per_model, on='model')
output_tokens_avg = output_tokens_avg.merge(thinking_tokens_avg, on='model')

output_tokens_avg

,model,Avg Output Tokens,thinking_mode,Avg Thinking Tokens
0,anthropic/claude-3.7-sonnet:thinking,360.250,True,8485.000
1,anthropic/claude-sonnet-4,390.250,False,0.000
2,deepseek/deepseek-chat-v3-0324,311.375,False,0.000
3,deepseek/deepseek-r1-0528,462.500,True,4642.125
4,google/gemini-2.5-flash-preview-05-20,517.625,False,0.000
5,google/gemini-2.5-flash-preview-05-20:thinking,454.000,True,5698.750
6,openai/gpt-4.1,381.625,False,0.000
7,openai/gpt-4o-mini,262.125,False,0.000
8,openai/o4-mini,344.250,True,4449.875
9,openai/o4-mini-high,294.250,True,12591.000


In [59]:
# Average cost per model
cost_avg = df.groupby('model')['cost'].mean().reset_index()
cost_avg.rename(columns={'cost': 'Avg Cost ($)'}, inplace=True)
cost_avg.sort_values("Avg Cost ($)")

,model,Avg Cost ($)
7,openai/gpt-4o-mini,0.000843
6,openai/gpt-4.1,0.000924
4,google/gemini-2.5-flash-preview-05-20,0.001060
2,deepseek/deepseek-chat-v3-0324,0.001599
5,google/gemini-2.5-flash-preview-05-20:thinking,0.004438
3,deepseek/deepseek-r1-0528,0.013214
1,anthropic/claude-sonnet-4,0.024573
8,openai/o4-mini,0.026178
9,openai/o4-mini-high,0.061741
0,anthropic/claude-3.7-sonnet:thinking,0.151549


In [52]:
# Extract company from model name
cost_avg['company'] = cost_avg['model'].str.split('/').str[0]

# Compute ratio per company
ratios = []

for company, group in cost_avg.groupby('company'):
    group_sorted = group.sort_values('Avg Cost ($)')
    
    if len(group_sorted) >= 2:
        cheaper = group_sorted.iloc[0]
        more_expensive = group_sorted.iloc[-1]
        
        ratio = more_expensive['Avg Cost ($)'] / cheaper['Avg Cost ($)']
        model_pair = f"{more_expensive['model']} vs {cheaper['model']}"
        
        ratios.append({'Company': company, 'Model Pair': model_pair, 'Cost Ratio': ratio})
    else:
        # Optionally, you can skip or report companies with only 1 model
        pass

# Convert to DataFrame
ratios_df = pd.DataFrame(ratios)
ratios_df


,Company,Model Pair,Cost Ratio
0,anthropic,anthropic/claude-3.7-sonnet:thinking vs anthropic/claude-sonnet-4,6.167287
1,deepseek,deepseek/deepseek-r1-0528 vs deepseek/deepseek-chat-v3-0324,8.263043
2,google,google/gemini-2.5-flash-preview-05-20:thinking vs google/gemini-2.5-flash-preview-05-20,4.185355
3,openai,openai/o4-mini vs openai/gpt-4.1,28.339430


In [53]:
# Average cost per model per attempt
cost_avg_attempt = df.groupby(['model', 'attempt'])[['cost', 'thinking_mode']].mean().reset_index()
cost_avg_attempt.rename(columns={'cost': 'Avg Cost ($)'}, inplace=True)
cost_avg_attempt.sort_values(['thinking_mode', 'model', 'attempt'])

,model,attempt,Avg Cost ($),thinking_mode
2,anthropic/claude-sonnet-4,1,0.023674,0.0
3,anthropic/claude-sonnet-4,2,0.025472,0.0
4,deepseek/deepseek-chat-v3-0324,1,0.001475,0.0
5,deepseek/deepseek-chat-v3-0324,2,0.001724,0.0
8,google/gemini-2.5-flash-preview-05-20,1,0.001005,0.0
9,google/gemini-2.5-flash-preview-05-20,2,0.001116,0.0
12,openai/gpt-4.1,1,0.000862,0.0
13,openai/gpt-4.1,2,0.000985,0.0
0,anthropic/claude-3.7-sonnet:thinking,1,0.209818,1.0
1,anthropic/claude-3.7-sonnet:thinking,2,0.093280,1.0


In [16]:
# Success rate per model compared across a few levels of tutorial verbosity
df['tutorial_len'] = df['tutorial'].apply(lambda x: len(x) if isinstance(x, str) else 0)

tutorial_success = df.groupby(['model', 'tutorial_len'])['success'].mean().reset_index()
tutorial_success['success'] = (tutorial_success['success'] * 100).round(2)
tutorial_success.rename(columns={'success': 'Success Rate (%)', 'tutorial_len': 'Tutorial Length'}, inplace=True)

tutorial_success

,model,Tutorial Length,Success Rate (%)
0,anthropic/claude-3.7-sonnet:thinking,8665,0.0
1,anthropic/claude-sonnet-4,8665,0.0
2,deepseek/deepseek-chat-v3-0324,8665,0.0
3,deepseek/deepseek-r1-0528,8665,0.0
4,google/gemini-2.5-flash-preview-05-20,8665,0.0
5,google/gemini-2.5-flash-preview-05-20:thinking,8665,0.0
6,openai/gpt-4o-mini,8665,0.0
7,openai/o4-mini-high,8665,0.0


In [80]:
# Success rate per model compared across a few levels of prompt verbosity
df['prompt_len'] = df['prompt'].apply(lambda x: len(x) if isinstance(x, str) else 0)

prompt_success = df.groupby(['model', 'prompt_len'])['success'].mean().reset_index()
prompt_success['success'] = (prompt_success['success'] * 100).round(2)
prompt_success.rename(columns={'success': 'Success Rate (%)', 'prompt_len': 'Prompt Length'}, inplace=True)

prompt_success

,model,Prompt Length,Success Rate (%)
0,openai/o4-mini,1294,18.18
1,x-ai/grok-3-mini-beta,1294,16.67


In [81]:
df_filtered = df[df['exercise'] == '6_8_1']
pd.set_option('display.max_colwidth', None)
df_filtered['errors'].to_frame()

,errors


In [56]:
df_filtered['line_with_error'].to_frame()

,line_with_error


In [82]:
import pandas as pd

# Group by run_id to get per-run summary
run_grouped = df.groupby('run_id').agg({
    'model': 'first',
    'success': list,
    'attempt': 'max',
    'max_attempts': 'first'
}).reset_index()

# Expand success list into per-k success map
max_k = df['max_attempts'].max()

for k in range(1, max_k + 1):
    def pass_at_k(row):
        successes = row['success']
        used_attempts = row['attempt']
        # Success in any of the first k attempts
        success_in_k = any(successes[:k])
        # Or the model didn't even use k attempts
        not_used_k = used_attempts < k
        return success_in_k or not_used_k

    run_grouped[f'pass@{k}'] = run_grouped.apply(pass_at_k, axis=1)

# Now compute per-model mean for each pass@k
passk_cols = [f'pass@{k}' for k in range(1, max_k + 1)]
passk_summary = run_grouped.groupby('model')[passk_cols].mean().reset_index()

# Convert to percentage
for col in passk_cols:
    passk_summary[col] = (passk_summary[col] * 100).round(2)

# Rename columns for display
passk_summary.rename(columns={col: f'Pass@{col[-1]} (%)' for col in passk_cols}, inplace=True)

passk_summary


,model,Pass@1 (%),Pass@2 (%),Pass@3 (%)
0,openai/o4-mini,25.0,50.0,50.0
1,x-ai/grok-3-mini-beta,50.0,50.0,50.0


In [ ]:
# TODO:
# thinking vs no thinking models (all)
# thinking vs no thinking on models that support both (fair)
# definition expanding comparison